In [28]:
import pandas as pd

df = pd.read_csv('../data/it_dataset.csv')
df = df[df['description'].apply(len)<2000]

df_train, df_test = df.iloc[:45000], df.iloc[45000:]

In [29]:
df_train['key_skills_list'] = df_train['key_skills'].str.split(';')

/tmp/ipykernel_401890/1578208901.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['key_skills_list'] = df_train['key_skills'].str.split(';')


In [41]:
all_key_skills = df_train['key_skills_list'].apply(pd.Series).stack().reset_index(drop=True)
all_key_skills_counts = all_key_skills.value_counts()

In [42]:
all_key_skills_counts = all_key_skills_counts[all_key_skills_counts>50]

In [43]:
df_test.description.iloc[0]

'Обязанности:  анализ документации и работа со статистикой; разработка нагрузочных скриптов (Java или C#) ; запуск тестов и анализ результатов, написание отчетов и составление документации; А также: разработка заглушек и эмуляторов, разработка утилит для генерации данных, работа с базами данных.  Требования:  уверенное знание одного из языков программирование: C/C++, Java, C#, Phython; умение работать с базами данных на уровне написания простейших SQL-запросов; знание архитектур приложений (толстый/тонкий клиент); возможность работать полный рабочий день.  Условия:  работа в молодом активном коллективе; конкурентная заработная плата, оформление по ТК РФ; постоянное обучение внутри компании; возможность профессионального роста; участие в тестировании ПО в крупных российских банках.    Первый этап - собеседование по онлайн.'

In [44]:
from flashtext.keyword import KeywordProcessor
keyword_processor = KeywordProcessor()
for key_skill in all_key_skills_counts.index:
    keyword_processor.add_keyword(key_skill)
    
keywords_found = keyword_processor.extract_keywords('Обязанности:  анализ документации и работа со статистикой; разработка нагрузочных скриптов (Java или C#) ; запуск тестов и анализ результатов, написание отчетов и составление документации; А также: разработка заглушек и эмуляторов, разработка утилит для генерации данных, работа с базами данных.  Требования:  уверенное знание одного из языков программирование: C/C++, Java, C#, Phython; умение работать с базами данных на уровне написания простейших SQL-запросов; знание архитектур приложений (толстый/тонкий клиент); возможность работать полный рабочий день.  Условия:  работа в молодом активном коллективе; конкурентная заработная плата, оформление по ТК РФ; постоянное обучение внутри компании; возможность профессионального роста; участие в тестировании ПО в крупных российских банках.    Первый этап - собеседование по онлайн.')
print(keywords_found)

['Java', 'C#', 'Работа с базами данных', 'программирование', 'C/C++', 'Java', 'C#', 'SQL', 'Обучение']


In [45]:
df_test['extracted_key_skills'] = df_test['description'].apply(lambda x: ';'.join(list(set(keyword_processor.extract_keywords(x)))))

/tmp/ipykernel_401890/3355085020.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['extracted_key_skills'] = df_test['description'].apply(lambda x: ';'.join(list(set(keyword_processor.extract_keywords(x)))))


In [46]:
df_test['extracted_key_skills']

73582    Работа с базами данных;C#;программирование;SQL...
73583    Контроль качества;Грамотная устная и письменна...
73585    Консультирование;1С;Удаленная работа;1С: Управ...
73588    ES6;docker;Web;JavaScript;vue;Docker-compose;П...
73589    Коммуникабельность; стрессоустойчивость;Высшее...
                               ...                        
83362    Уверенный пользователь ПК;Грамотная устная и п...
83368    Поиск и привлечение клиентов;Высокие коммуника...
83371    VueJS;ООП;Yii2;Дизайнер;Node.js;React/Redux;Ра...
83373                                Компьютер;1С;Ethernet
83374    Опытный пользователь ПК;Высшее образование;MPL...
Name: extracted_key_skills, Length: 5705, dtype: object

In [47]:
df_test.to_csv('../data/it_dataset_test_search.csv', index = False)